In [64]:
import cv2
import numpy as np
import time
import os
from matplotlib import pyplot as plt
import mediapipe as mp

KEYPOINTS USING MP HOLISTIC

In [65]:
mp_holistic= mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils

In [66]:
def mediapipe_detection(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results

In [67]:

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [68]:
def draw_styled_landmarks(image,results):
     mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                               mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                               mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                               mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=3),
                               mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
     mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                               mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=3),
                               mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
     mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                               mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=3),
                               mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

In [69]:
cap=cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        #make detections
        image, results= mediapipe_detection(frame, holistic)
        print(results)
        draw_styled_landmarks(image,results)
        #show to screen
        cv2.imshow('OpenCV feed',image)
        # break 
        if cv2.waitKey(10) & 0XFF==ord('q'):
             break
cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

q

In [16]:
len(results.face_landmarks.landmark)

468

In [17]:
results

mediapipe.python.solution_base.SolutionOutputs

In [18]:
draw_landmarks(frame, results)

EXTRACT KEYPOINT VALUES

In [22]:
pose=[]
for res in results.pose_landmarks.landmark:
    test=np.array([res.x,res.y,res.z,res.visibility])
    pose.append(test)

In [23]:
pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [25]:
len(results.face_landmarks.landmark)

468

In [26]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh,rh])

In [59]:
extract_keypoints(results).shape

(1662,)

SETUP FOLDER FOR COLLECTION

In [29]:
#path for exported data, numpy arrays
DATA_PATH=os.path.join('MP_data')
# actions that we try to detect
actions=np.array(['help','hello','i need help'])
#fourty videos data
no_sequences=30
#videos frame length
sequence_lenght=30

In [31]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action,str(sequence)))
        except:
            pass

In [32]:
result_test=extract_keypoints(results)

In [33]:
np.save('0',result_test)

In [34]:
np.load('0.npy')

array([ 0.70170665,  0.75522822, -0.91112077, ...,  0.        ,
        0.        ,  0.        ])

In [65]:
result_test

array([ 0.49792486,  0.44073763, -1.01808918, ...,  0.        ,
        0.        ,  0.        ])

COLLECT KEYPOINT VALUES FOR TRAINING AND TESTING

In [35]:
cap=cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_lenght):
                ret,frame=cap.read()
              #make detections
                image, results= mediapipe_detection(frame, holistic)
                print(results)
                draw_styled_landmarks(image,results)
                
                #apply wait logic
                if frame_num==0:
                  cv2.putText(image, 'STARTING COLLECTION',(120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                  cv2.putText(image, 'Collecting frames for{} Video Number{}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                  cv2.waitKey(30)
                else:
                  cv2.putText(image, 'Collecting frames for{} Video Number{}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
        #show to screen
                  cv2.imshow('OpenCV feed',image)
                #new export keypoints
                keypoints=extract_keypoints(results)
                npy_path=os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)
                # break 
                if cv2.waitKey(10) & 0XFF==ord('q'):
                  break
cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

PREPROCESS DATA AND CREATE LABELS AND FEATURES

In [36]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [37]:
label_map={label:num for num,label in enumerate(actions)}

In [38]:
label_map

{'help': 0, 'hello': 1, 'i need help': 2}

In [39]:
sequences,labels=[],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_lenght):
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [40]:
x=np.array(sequences)

In [41]:
y=to_categorical(labels).astype(int)

In [42]:
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.05)

In [43]:
y_test.shape

(5, 3)

build and train lstm nerural network

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import os
import numpy as np

# # Define the possible actions (output classes)
# actions = np.array(['action1', 'action2', 'action3'])  # Example actions

# Define the log directory for TensorBoard
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
# Build the model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))  # Output layer
# Compile the model (optional)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\Pranshu Agrahari\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [45]:
model.fit(x_train,y_train,epochs=2000,callbacks=[tb_callback])

Epoch 1/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 124ms/step - accuracy: 0.4011 - loss: 24.2765
Epoch 2/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.3542 - loss: 8.7430
Epoch 3/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.3327 - loss: 22.8782
Epoch 4/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.3778 - loss: 5.6595
Epoch 5/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.3719 - loss: 5.3703
Epoch 6/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.3816 - loss: 7.0453
Epoch 7/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.2564 - loss: 5.6241
Epoch 8/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.2721 - loss: 10.4040
Epoch 9/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.4090 - loss: 4.9836
Epoch 10/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.4618 - loss: 3.7741
Epoch 11/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.3718 - loss: 2.7238
Epoch 12/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/st

MAKE PREDICTIONS


In [52]:
model.predict(x_test)
res=model.predict(x_test)
actions[np.argmax(res[4])]
actions[np.argmax(y_test[2])]
model.save('action.h5')
model.load_weights('action.h5')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


Save Weighhts

Evaluation using Confusion Matrix And Accuracy

In [53]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat=model.predict(x_test)
ytrue= np.argmax(y_test,axis=1).tolist()
yhat=np.argmax(yhat,axis=1).tolist()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [54]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[0., 0.],
        [5., 0.]],

       [[0., 5.],
        [0., 0.]]])

In [55]:
accuracy_score(ytrue,yhat)

0.0

Test in Real Time

In [57]:
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    
    for num, prob in enumerate(res):
        # Draw rectangles based on probabilities
        cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        
        # Add action text to the frame
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return output_frame

In [58]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res,actions,image,colors))

TypeError: only length-1 arrays can be converted to Python scalars

<Figure size 1800x1800 with 0 Axes>

In [59]:
from twilio.rest import Client
import geocoder


In [60]:


#setup twilio for SMS alert
account_sid = "ACb193b19d90b378949907dd4895ad7c4c"
auth_token = "dd77ef752992bedddc8878508f9fbf7d"
client = Client(account_sid,auth_token)

#Function to send SmS alerts
def send_alert_message(action,phone_number):
    location = geocoder.ip('me').latlng
    message_body = f"Action detected: {action}. Device location: {location[0]}, {location[1]}"
    message = client.messages.create(body=message_body,from_='+14698888251', to=phone_number)
    print(f"Alert sent: {message_body}")
    return message.sid

In [62]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
sequence = []
sentence = []
predictions = []
threshold = 0.4
#load your trained model
model= load_model("action.h5")
cap=cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        #make detections
        image, results= mediapipe_detection(frame, holistic)
        print(results)
        draw_styled_landmarks(image,results)
        # prediction logic
        keypoints=extract_keypoints(results)
        sequence.insert(0,keypoints)
        sequence=sequence[:30]
        
        if len(sequence)==30:
            res=model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])
            
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
                    
                # Send alert if a specific action is detected
                detected_action = actions[np.argmax(res)]
                if detected_action in ['i need help']:  # Replace 'help' with any sign that needs alert
                    send_alert_message(detected_action, '+919335402749')    
                            
                    
        if len(sentence) > 5:
            sentence=sentence[-5:]
        
        #viz probabilities
        image=prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image,(0,0),(640,40),(245,117,16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                      
        #show to screen
        cv2.imshow('OpenCV feed',image)
        # break 
        if cv2.waitKey(10) & 0XFF==ord('q'):
             break
cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>


TypeError: only length-1 arrays can be converted to Python scalars

In [63]:
res[np.argmax(res)] >threshold

array([False, False, False])